In [1]:
import skopt
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.svm import SVC
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize

In [2]:
# Test!!

url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
df = pd.read_csv(url, header=None)
data = df.values

x, y = data[:, :-1], data[:, -1]

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEquidistantSelection(TransformerMixin, BaseEstimator):
    def __init__(self, n_features_to_select=None):
        self.n_features_to_select = n_features_to_select
        self.selected_features_idx = []

    def get_params(self, deep=True):
        return {"n_features_to_select": self.n_features_to_select}       

    def fit(self, X, y=None):
        n_features = X.shape[1]
        self.selected_features_idx = np.linspace(0, n_features-1, self.n_features_to_select, dtype=int)
        return self

    def transform(self, X, y=None, **kwargs):
        return X[:, self.selected_features_idx]

In [6]:
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV


pipe = Pipeline([("transform", FeatureEquidistantSelection()), ('svc', SVC())])

params = dict()
params['transform__n_features_to_select'] = (8, 34, 'uniform')
params['svc__C'] = (1e-6, 100.0, 'log-uniform')
params['svc__gamma'] = (1e-6, 100.0, 'log-uniform')
params['svc__degree'] = (1,5)
params['svc__kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the search
search = BayesSearchCV(estimator=pipe, search_spaces=params, n_jobs=-1, cv=cv)
# perform the search
search.fit(x, y)
# report the best result
print(search.best_score_)
print(search.best_params_)

0.9534662867996201
OrderedDict([('svc__C', 11.078210467053394), ('svc__degree', 5), ('svc__gamma', 0.04932410479000081), ('svc__kernel', 'rbf'), ('transform__n_features_to_select', 34)])
